In [8]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import itertools
import random
import pyomo
import time
# A, b, c
A = np.array([[1,0,1,0,1,0,-1,0,0,0],[0,1,0,1,0,1,0,-1,0,0],[0,0,2,1,0,0,0,0,1,0],[0,0,1,2,0,0,0,0,0,1]])
c = np.array([35,40,16,19,47,54,0,0,0,0])

In [9]:
Matrix(A)

Matrix([
[1, 0, 1, 0, 1, 0, -1,  0, 0, 0],
[0, 1, 0, 1, 0, 1,  0, -1, 0, 0],
[0, 0, 2, 1, 0, 0,  0,  0, 1, 0],
[0, 0, 1, 2, 0, 0,  0,  0, 0, 1]])

In [10]:
# Computation of Graver Basis

In [11]:
# toric ideal of A

In [12]:
def toric_ideal(A):
    # Define symbolic variables ys for each row (index 0 in Python)
    sym_str_y = 'y:' + str(A.shape[0])
    ys = symbols(sym_str_y)
    
    # Define symbolic variables xs for each column (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)

    def to_polynomial(coef,vars):
        '''
        Function to define a single column of the coefficient as a polynomial
        '''
        res1 = 1
        res2 = 1
        for i in range(len(coef)):
            if coef[i] >= 0:
                res1 = res1*vars[i]**coef[i]
            else:
                res2 = res2*vars[i]**(-coef[i])
        res = res1 - res2
        return res

    def polynomial_ideal(A):
        '''
        Function to define a the polynomial ideal of a matrix A according to Conti and Traverso
        '''
        IA = A.col_insert(0, eye(A.shape[0]))
        # Find nullspace (kernel) of A
        ker = IA.nullspace()

        # Normalize elements of kernel to be integers
        ker_len = len(ker)
        for i in range(ker_len):
            rationalvector = True
            while rationalvector:
                factor = 1
                for j in ker[i]:
                    if j%1 != 0:
                        factor = min(factor,j%1)
                if factor == 1:
                    rationalvector = False
                else:
                    ker[i]=ker[i] / factor

        vars = ys + xs

        gen = []
        for k in ker:
            gen.append(to_polynomial(k,vars))


        return(gen, vars)
    
    IA, vars = polynomial_ideal(A)
    tor = groebner(IA, vars, order='lex')

    toric = []

    for i in tor:
        i = Poly(i)
        i_str = str(i.gens)
        #print(i_str)
        if not 'y' in i_str:
            toric.append(i)

    return toric, xs, ys

In [13]:
# Graver Basis of A

# Main

In [14]:
def GraverBasis(A):

    def Alaw(A):
        # n : column dimension r : row dimension
        A = Matrix(A)
        r = A.shape[0]
        n = A.shape[1]
        Id = np.concatenate((np.identity(n),np.identity(n)),axis = 1)
        Alaw = np.concatenate((A, np.zeros((r, n))),axis = 1)
        Alaw = np.concatenate((Alaw, Id),axis = 0)

        Afin = Alaw.astype(int)
        Afin = Matrix(Afin)
        return Afin, n

    def monomial(p):
        return [prod(x**k for x, k in zip(p.gens, mon)) for mon in p.monoms()]

    def to_T(toric):
        toric_fin=[]
        for g in toric:
            for k in range(n,2*n):
                g = g.subs({(xs[k],1)})
            toric_fin.append(g)

        toric_len = len(toric)

        vp = [0]*n
        vm = [0]*n
        T = []
        for k in range(0,toric_len):
            for i in range(0,n):
                p = monomial(Poly(toric_fin[k]))[0]
                m = monomial(Poly(toric_fin[k]))[1]
                vp[i] = degree(p,xs[i])
                vm[i] = degree(m,xs[i])
            v = np.array(vp) - np.array(vm)
            v = v.astype(int)
            T.append(v)
        return T

    Afin, n = Alaw(A)
    toric, xs, ys= toric_ideal(Afin)

    T = to_T(toric)
    return T


# Aungmentation Algrithm

In [15]:
# augmentation algorithm

In [23]:
def augmentation(z_feas,c,T):
    # z_feaas: feasible point ; c: cost; T: universal test set
    exist_aug = True
    #i = 0
    while exist_aug:
        exist_aug = False
        for t in T:
            if np.dot(c, t, out=None)>0 and np. all((z_feas-t>=0)):
                z_feas = z_feas-t
                #i = 1+i
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
            if np.dot(c, t, out=None)<0 and np. all((z_feas+t>=0)):
                z_feas = z_feas+t
                #i = i+1
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
    #print('Achieve an optimum!')
    return z_feas

# Experiment

In [17]:
def feasible_solu(b):
    return [0,0,0,0,b[0],b[1],0,0,b[2],b[3]]

In [18]:
start_time = time.time()

T = GraverBasis(A)

print("--- %s seconds ---" % (time.time() - start_time))

--- 131.8309199810028 seconds ---


In [38]:
#T

start_time = time.time()

opt = []
for i in range(0,600,3):
    b = [300+i,300+i,200+10*i,200+10*i]
    temp = augmentation(feasible_solu(b),c,T)
    opt.append(temp)

#opt
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.8873610496521 seconds ---


In [21]:
#T